
---

# ⚙️ ModelConfig

### 🎯 Intent

Use **`model_config`** to control a model’s **global behavior** in Pydantic v2 — covering validation, serialization, schema, and field handling.

---

### 🧩 Core Components

1. **🧩 Central Config (`ConfigDict`)**

   * Define all model-wide rules in one place.
   * Common keys: `title`, `strict`, `extra`, `json_schema_extra`.

2. **🔒 Strict Mode**

   * `strict=True` → no automatic type coercion.
   * Useful for APIs and financial data where exact input types matter.

3. **🧱 Extra Fields Policy**

   * `extra="ignore"` → drop unknown keys.
   * `extra="allow"` → keep them.
   * `extra="forbid"` → raise error on unexpected fields.

4. **🏷️ Aliases & Naming**

   * `populate_by_name=True` → allow both field name and alias in input/output.
   * Helpful for stable external API contracts.

5. **📤 Serialization Defaults**

   * Options for handling JSON (`inf/nan`, `timedelta`).
   * `json_schema_extra` → add OpenAPI metadata (e.g., descriptions, ordering).

6. **🧪 Validation Controls**

   * `validate_default=True|False` → toggle validation of default values.
   * `from_attributes=True` → read from object attributes (like v1 `orm_mode`).
   * `use_attribute_docstrings=True` → auto-use docstrings for schema descriptions.

7. **🧰 Inheritance**

   * Child models inherit parent config.
   * Can override with their own `ConfigDict`.

8. **🧠 Common Presets**

   * **Strict API mode:** `strict=True`, `extra="forbid"`.
   * **ORM/DB mode:** `from_attributes=True`.
   * **Public JSON mode:** add `json_schema_extra`, `populate_by_name=True`.

---


In [1]:
from pydantic import BaseModel, ConfigDict, Field
from typing import Annotated

class User(BaseModel):
    # ✅ Apply global config
    model_config = ConfigDict(
        strict=True,             # no type coercion
        extra="forbid",          # forbid unknown fields
        populate_by_name=True,   # allow alias or field name
        from_attributes=True,    # ORM/attribute mode
        json_schema_extra={"description": "User schema for API"}
    )

    id: int
    username: Annotated[str, Field(alias="user_name")]
    email: str
    age: int = 18  # default value (validated if validate_default=True)

# 1️⃣ Valid input (alias accepted)
user = User(user_name="Mukesh", id=1, email="mukesh@example.com")
print(user.model_dump())  # {'id': 1, 'username': 'Mukesh', 'email': 'mukesh@example.com', 'age': 18}

# 2️⃣ Invalid type (strict=True → no coercion)
try:
    User(id="2", user_name="Test", email="test@example.com")
except Exception as e:
    print("Strict mode error:", e)

# 3️⃣ Extra field (forbid → error)
try:
    User(id=3, user_name="Extra", email="e@example.com", extra_field="oops")
except Exception as e:
    print("Extra field error:", e)


{'id': 1, 'username': 'Mukesh', 'email': 'mukesh@example.com', 'age': 18}
Strict mode error: 1 validation error for User
id
  Input should be a valid integer [type=int_type, input_value='2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_type
Extra field error: 1 validation error for User
extra_field
  Extra inputs are not permitted [type=extra_forbidden, input_value='oops', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden
